# 2. Stereo Vision Calibration

Reads the image pairs saved by `ag-cam-tools calibration-capture` from
`stereoLeft/` and `stereoRight/`, detects checkerboard corners in each pair,
and runs OpenCV stereo calibration.

Outputs intrinsics, extrinsics, and rectification maps to `calib_result/`
for use in `3.Depthmap_with_Tuning_Bar.ipynb`.

**This notebook has no camera dependency** --- all processing is offline.

---
**Camera: Lucid PHD IMX273 --- 40 mm baseline, 3 mm FL**

Image resolution is auto-detected from the captured files (supports both
full resolution 1440x1080 and 2:1 binned 720x540).

The baseline distance is not set here; it is computed from the calibration
images and stored in the exported extrinsics (translation vector `T`).

### Setup (one-time)
```bash
cd calibration
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user --name agrippa-calibration --display-name "agrippa-calibration"
```

Then select the **agrippa-calibration** kernel in Jupyter before running.

## Dependencies

In [ ]:
%pip install -r requirements.txt -q

In [1]:
import glob
import os
import cv2
import numpy as np

## Configuration

If you used a different checkerboard, update `rows`, `columns`, and
`square_size` to match.

Image size is auto-detected from the first loaded image.

In [ ]:
# Checkerboard inner-corner count and physical square size (cm).
# Inner corners = squares - 1 in each dimension.
# For an 18x25 square board, inner corners are 17x24.
rows        = 17
columns     = 24
square_size = 0.75  # cm

# 3D object points for one board view (Z=0 plane).
objp = np.zeros((rows * columns, 3), np.float32)
objp[:, :2] = np.indices((rows, columns)).T.reshape(-1, 2)
objp *= square_size

# Subpixel refinement criteria.
corner_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.01)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Scan for calibration session folders (contain stereoLeft/).
_sessions = sorted(
    [d for d in os.listdir(".")
     if os.path.isdir(d) and d.startswith("calibration_")
     and os.path.isdir(os.path.join(d, "stereoLeft"))],
    reverse=True,  # most recent first
)

if not _sessions:
    raise FileNotFoundError(
        "No calibration session folders found in the current directory.\n"
        "Run `ag-cam-tools calibration-capture` first."
    )

_session_dropdown = widgets.Dropdown(
    options=_sessions,
    value=_sessions[0],
    description="Session:",
    layout=widgets.Layout(width="500px"),
    style={"description_width": "80px"},
)

# Count images in each session for the info label.
def _session_info(name):
    n = len(glob.glob(os.path.join(name, "stereoLeft", "*.png")))
    has_calib = os.path.isdir(os.path.join(name, "calib_result"))
    status = " (calibrated)" if has_calib else ""
    return f"{n} image pairs{status}"

_info_label = widgets.Label(value=_session_info(_sessions[0]))

def _on_change(change):
    _info_label.value = _session_info(change["new"])

_session_dropdown.observe(_on_change, names="value")
display(widgets.HBox([_session_dropdown, _info_label]))

# Downstream cells read from this variable.
session_path = _session_dropdown.value

## Corner detection pass

Each image pair is loaded, then OpenCV searches for checkerboard corners
with subpixel refinement. Pairs where the board is not fully visible in
both frames are automatically discarded.

In [ ]:
object_points     = []
image_points_left = []
image_points_right = []
accepted = 0
discarded = 0

# Read the current dropdown selection (in case it changed since the picker cell ran).
session_path = _session_dropdown.value

# Resolve session path and create calib_result inside it.
session_abs = os.path.abspath(session_path)
calib_result_path = os.path.join(session_abs, "calib_result")
os.makedirs(calib_result_path, exist_ok=True)

left_glob  = os.path.join(session_abs, "stereoLeft",  "*.png")
right_glob = os.path.join(session_abs, "stereoRight", "*.png")

images_left  = sorted(glob.glob(left_glob))
images_right = sorted(glob.glob(right_glob))

if not images_left:
    raise FileNotFoundError(
        f"No images found in {left_glob}. "
        "Run `ag-cam-tools calibration-capture` first, then set session_path above."
    )

# Auto-detect image size from the first image.
first_img = cv2.imread(images_left[0])
image_size = (first_img.shape[1], first_img.shape[0])  # (width, height)

print(f"Session:    {session_abs}")
print(f"Found {len(images_left)} left / {len(images_right)} right images.")
print(f"Image size: {image_size[0]} x {image_size[1]}")
print("Starting corner detection...\n")

for img_left_path, img_right_path in zip(images_left, images_right):
    print(f"Pair {accepted + discarded:03d}  {os.path.basename(img_left_path)}")

    img_left  = cv2.imread(img_left_path,  cv2.IMREAD_UNCHANGED)
    img_right = cv2.imread(img_right_path, cv2.IMREAD_UNCHANGED)

    # Ensure both images are BGR.
    if img_left.ndim == 2:
        img_left  = cv2.cvtColor(img_left,  cv2.COLOR_GRAY2BGR)
        img_right = cv2.cvtColor(img_right, cv2.COLOR_GRAY2BGR)

    img_left  = cv2.resize(img_left,  image_size)
    img_right = cv2.resize(img_right, image_size)

    # Detect corners in both images.
    gray_l = cv2.cvtColor(img_left,  cv2.COLOR_BGR2GRAY)
    gray_r = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    ret_l, corners_l = cv2.findChessboardCorners(gray_l, (rows, columns))
    ret_r, corners_r = cv2.findChessboardCorners(gray_r, (rows, columns))

    if not ret_l or not ret_r:
        reason = "left" if not ret_l else "right"
        if not ret_l and not ret_r:
            reason = "both"
        print(f"  -> discarded: chessboard not found in {reason}")
        discarded += 1
        continue

    # Subpixel refinement.
    cv2.cornerSubPix(gray_l, corners_l, (11, 11), (-1, -1), corner_criteria)
    cv2.cornerSubPix(gray_r, corners_r, (11, 11), (-1, -1), corner_criteria)

    object_points.append(objp)
    image_points_left.append(corners_l.reshape(-1, 2))
    image_points_right.append(corners_r.reshape(-1, 2))
    accepted += 1

print(f"\nDone. {accepted} pairs accepted, {discarded} discarded.")

## Per-camera intrinsic calibration

Run `cv2.calibrateCamera` independently on each camera before the joint
stereo solve.  This gives per-camera RMS reprojection errors (useful for
spotting a bad lens or dataset) and provides a good initial guess for the
more constrained `cv2.stereoCalibrate` that follows.

In [ ]:
calib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
mono_flags = (cv2.CALIB_FIX_ASPECT_RATIO |
              cv2.CALIB_ZERO_TANGENT_DIST |
              cv2.CALIB_RATIONAL_MODEL)

rms_l, cam_mat_l_init, dist_l_init, _, _ = cv2.calibrateCamera(
    object_points, image_points_left, image_size,
    cameraMatrix=None, distCoeffs=None,
    criteria=calib_criteria, flags=mono_flags)

rms_r, cam_mat_r_init, dist_r_init, _, _ = cv2.calibrateCamera(
    object_points, image_points_right, image_size,
    cameraMatrix=None, distCoeffs=None,
    criteria=calib_criteria, flags=mono_flags)

print(f"Left  camera RMS: {rms_l:.4f} px")
print(f"Right camera RMS: {rms_r:.4f} px")
print(f"Left  focal length: fx={cam_mat_l_init[0,0]:.1f}  fy={cam_mat_l_init[1,1]:.1f} px")
print(f"Right focal length: fx={cam_mat_r_init[0,0]:.1f}  fy={cam_mat_r_init[1,1]:.1f} px")

if abs(rms_l - rms_r) > 0.3:
    print(f"\n  Warning: large RMS difference between cameras ({abs(rms_l - rms_r):.2f} px).")
    print("  Check for focus mismatch or a damaged lens.")

## Stereo calibration

Runs `cv2.stereoCalibrate` seeded with the per-camera intrinsics estimated
above (`CALIB_USE_INTRINSIC_GUESS`).  This constrains the joint solver and
improves convergence, especially with the 8-parameter rational distortion
model.

A reprojection error below **0.5 px** indicates a good calibration.

> **Distortion model:** the Edmund Optics #20-061 lens has 34.78% barrel
> distortion at full field. The rational model (`cv2.CALIB_RATIONAL_MODEL`)
> is used below, which fits an 8-coefficient distortion model instead of the
> default 5. This better captures the heavy radial distortion of these
> wide-angle lenses.

In [ ]:
print("Starting stereo calibration...")

stereo_flags = (cv2.CALIB_FIX_ASPECT_RATIO |
                cv2.CALIB_ZERO_TANGENT_DIST |
                cv2.CALIB_SAME_FOCAL_LENGTH |
                cv2.CALIB_RATIONAL_MODEL |
                cv2.CALIB_USE_INTRINSIC_GUESS)

rms, cam_mat_l, dist_l, cam_mat_r, dist_r, R, T, E, F = cv2.stereoCalibrate(
    object_points,
    image_points_left,
    image_points_right,
    cameraMatrix1=cam_mat_l_init.copy(), distCoeffs1=dist_l_init.copy(),
    cameraMatrix2=cam_mat_r_init.copy(), distCoeffs2=dist_r_init.copy(),
    imageSize=image_size,
    criteria=calib_criteria,
    flags=stereo_flags)

print(f"Stereo RMS reprojection error: {rms:.4f} px")
print(f"Distortion coefficients: {dist_l.shape[1]} parameters (rational model)")
print(f"Baseline (||T||): {np.linalg.norm(T):.2f} cm")
if rms > 0.5:
    print("\n  Warning: error > 0.5 px — consider re-capturing images from more angles.")
else:
    print("  Error within acceptable range.")

## Per-pair reprojection error analysis

Project each pair's object points through the stereo model and measure the
reprojection error.  Pairs with RMS significantly above the median are
likely bad captures (motion blur, partial occlusion, pattern flex).

If outliers are flagged, the next cell removes them — then re-run the
**Per-camera intrinsic calibration** and **Stereo calibration** cells above
to get a cleaner result.

In [ ]:
import matplotlib.pyplot as plt

# Re-derive per-image poses via solvePnP on the left camera (reference frame),
# then transform to the right camera using R, T from stereoCalibrate.
per_pair_rms_l = []
per_pair_rms_r = []
for i in range(len(object_points)):
    pts_3d = object_points[i]
    pts_2d_l = image_points_left[i]
    pts_2d_r = image_points_right[i]

    _, rvec_l, tvec_l = cv2.solvePnP(pts_3d, pts_2d_l, cam_mat_l, dist_l)

    # Right camera pose = R @ left_pose + T.
    rvec_r, _ = cv2.Rodrigues(R @ cv2.Rodrigues(rvec_l)[0])
    tvec_r = R @ tvec_l + T

    proj_l, _ = cv2.projectPoints(pts_3d, rvec_l, tvec_l, cam_mat_l, dist_l)
    proj_r, _ = cv2.projectPoints(pts_3d, rvec_r, tvec_r, cam_mat_r, dist_r)

    err_l = np.linalg.norm(pts_2d_l - proj_l.reshape(-1, 2), axis=1)
    err_r = np.linalg.norm(pts_2d_r - proj_r.reshape(-1, 2), axis=1)

    per_pair_rms_l.append(np.sqrt(np.mean(err_l**2)))
    per_pair_rms_r.append(np.sqrt(np.mean(err_r**2)))

per_pair_rms_l = np.array(per_pair_rms_l)
per_pair_rms_r = np.array(per_pair_rms_r)
per_pair_rms_combined = np.sqrt((per_pair_rms_l**2 + per_pair_rms_r**2) / 2)

median_rms = np.median(per_pair_rms_combined)
outlier_threshold = max(2.0 * median_rms, 1.0)  # at least 1.0 px
outlier_mask = per_pair_rms_combined > outlier_threshold

# Print table.
print(f"{'Pair':>4}  {'Left RMS':>9}  {'Right RMS':>10}  {'Combined':>9}  {'Status'}")
print("-" * 55)
for i in range(len(per_pair_rms_combined)):
    flag = " ** OUTLIER **" if outlier_mask[i] else ""
    print(f"{i:4d}  {per_pair_rms_l[i]:9.3f}  {per_pair_rms_r[i]:10.3f}  "
          f"{per_pair_rms_combined[i]:9.3f}{flag}")

n_outliers = int(outlier_mask.sum())
print(f"\nMedian RMS: {median_rms:.3f} px | Outlier threshold: {outlier_threshold:.3f} px")
print(f"Outliers: {n_outliers} / {len(per_pair_rms_combined)}")

# Bar chart.
fig, ax = plt.subplots(figsize=(max(8, len(per_pair_rms_combined) * 0.4), 4))
colors = ["tab:red" if m else "tab:blue" for m in outlier_mask]
ax.bar(range(len(per_pair_rms_combined)), per_pair_rms_combined, color=colors)
ax.axhline(y=outlier_threshold, color="red", linestyle="--", linewidth=0.8,
           label=f"threshold ({outlier_threshold:.2f} px)")
ax.axhline(y=median_rms, color="gray", linestyle=":", linewidth=0.8,
           label=f"median ({median_rms:.2f} px)")
ax.set_xlabel("Image pair index")
ax.set_ylabel("RMS reprojection error (px)")
ax.set_title("Per-pair reprojection error")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Remove outlier pairs from the point lists.
# After running this cell, re-run "Per-camera intrinsic calibration" and
# "Stereo calibration" above to recalibrate without the bad pairs.

if n_outliers == 0:
    print("No outliers detected — nothing to remove.")
else:
    keep = ~outlier_mask
    object_points     = [object_points[i]     for i in range(len(keep)) if keep[i]]
    image_points_left = [image_points_left[i]  for i in range(len(keep)) if keep[i]]
    image_points_right = [image_points_right[i] for i in range(len(keep)) if keep[i]]
    print(f"Removed {n_outliers} outlier pair(s). {len(object_points)} pairs remain.")
    print("Re-run the Per-camera intrinsic calibration and Stereo calibration cells above.")

## Stereo rectification and export

Compute rectification transforms, undistortion/rectification maps, and
export all calibration artifacts as `.npy` files to `calib_result/`.

In [ ]:
# Read the current dropdown selection (in case it changed since the picker cell ran).
session_path = _session_dropdown.value
session_abs = os.path.abspath(session_path)
calib_result_path = os.path.join(session_abs, "calib_result")
os.makedirs(calib_result_path, exist_ok=True)

# Stereo rectification.
R1, R2, P1, P2, Q, valid_roi_l, valid_roi_r = cv2.stereoRectify(
    cam_mat_l, dist_l,
    cam_mat_r, dist_r,
    image_size, R, T,
    flags=0)

# Compute undistortion + rectification maps.
map_l1, map_l2 = cv2.initUndistortRectifyMap(
    cam_mat_l, dist_l, R1, P1, image_size, cv2.CV_32FC1)
map_r1, map_r2 = cv2.initUndistortRectifyMap(
    cam_mat_r, dist_r, R2, P2, image_size, cv2.CV_32FC1)

# Export all calibration results as .npy files (compatible with notebook 3).
np.save(os.path.join(calib_result_path, "cam_mats_left.npy"),           cam_mat_l)
np.save(os.path.join(calib_result_path, "cam_mats_right.npy"),          cam_mat_r)
np.save(os.path.join(calib_result_path, "dist_coefs_left.npy"),         dist_l)
np.save(os.path.join(calib_result_path, "dist_coefs_right.npy"),        dist_r)
np.save(os.path.join(calib_result_path, "rot_mat.npy"),                 R)
np.save(os.path.join(calib_result_path, "trans_vec.npy"),               T)
np.save(os.path.join(calib_result_path, "e_mat.npy"),                   E)
np.save(os.path.join(calib_result_path, "f_mat.npy"),                   F)
np.save(os.path.join(calib_result_path, "rect_trans_left.npy"),         R1)
np.save(os.path.join(calib_result_path, "rect_trans_right.npy"),        R2)
np.save(os.path.join(calib_result_path, "proj_mats_left.npy"),          P1)
np.save(os.path.join(calib_result_path, "proj_mats_right.npy"),         P2)
np.save(os.path.join(calib_result_path, "disp_to_depth_mat.npy"),       Q)
np.save(os.path.join(calib_result_path, "valid_boxes_left.npy"),        np.array(valid_roi_l))
np.save(os.path.join(calib_result_path, "valid_boxes_right.npy"),       np.array(valid_roi_r))
np.save(os.path.join(calib_result_path, "undistortion_map_left.npy"),   map_l1)
np.save(os.path.join(calib_result_path, "undistortion_map_right.npy"),  map_r1)
np.save(os.path.join(calib_result_path, "rectification_map_left.npy"),  map_l2)
np.save(os.path.join(calib_result_path, "rectification_map_right.npy"), map_r2)

print(f"Calibration exported to {calib_result_path}/ ({len(os.listdir(calib_result_path))} files)")
print(f"Rectified focal length: {P1[0,0]:.1f} px")
print(f"Baseline (||T||): {np.linalg.norm(T):.2f} cm")

## Rectification quality — epipolar error

The key invariant for stereo disparity: after rectification, corresponding
points must lie on the **same horizontal scanline** in both images.

This cell undistorts and rectifies every detected corner pair, then
measures the absolute y-difference.  A mean epipolar error below **0.5 px**
is needed for reliable disparity matching; above **1.0 px** will cause
visible artefacts.

In [ ]:
all_y_errors = []

for i in range(len(object_points)):
    pts_l = image_points_left[i].reshape(-1, 1, 2).astype(np.float64)
    pts_r = image_points_right[i].reshape(-1, 1, 2).astype(np.float64)

    # Undistort + rectify corner positions.
    rect_l = cv2.undistortPoints(pts_l, cam_mat_l, dist_l, R=R1, P=P1)
    rect_r = cv2.undistortPoints(pts_r, cam_mat_r, dist_r, R=R2, P=P2)

    y_err = np.abs(rect_l[:, 0, 1] - rect_r[:, 0, 1])
    all_y_errors.append(y_err)

all_y_errors = np.concatenate(all_y_errors)

mean_epipolar = np.mean(all_y_errors)
max_epipolar = np.max(all_y_errors)
std_epipolar = np.std(all_y_errors)

print(f"Epipolar error (y-difference after rectification):")
print(f"  Mean: {mean_epipolar:.3f} px")
print(f"  Max:  {max_epipolar:.3f} px")
print(f"  Std:  {std_epipolar:.3f} px")
print(f"  Points measured: {len(all_y_errors)}")

if mean_epipolar > 1.0:
    print("\n  FAIL: mean epipolar error > 1.0 px — disparity will be unreliable.")
    print("  Re-capture calibration images or investigate outliers above.")
elif mean_epipolar > 0.5:
    print("\n  Warning: mean epipolar error > 0.5 px — disparity quality may suffer.")
else:
    print("\n  Rectification quality is good for disparity matching.")

# Histogram.
fig, ax = plt.subplots(figsize=(8, 3))
ax.hist(all_y_errors, bins=50, edgecolor="black", linewidth=0.3)
ax.axvline(x=mean_epipolar, color="red", linestyle="--", linewidth=1,
           label=f"mean ({mean_epipolar:.3f} px)")
ax.axvline(x=0.5, color="orange", linestyle=":", linewidth=1, label="0.5 px target")
ax.set_xlabel("Absolute y-error (px)")
ax.set_ylabel("Count")
ax.set_title("Epipolar error distribution (all rectified corners)")
ax.legend()
plt.tight_layout()
plt.show()

## Export C-ready remap tables

Converts the float32 rectification maps into pre-computed pixel-offset
lookup tables that `ag-cam-tools stream --rectify` can load directly.

Each `.bin` file contains a 16-byte header (`RMAP` magic + width + height +
flags) followed by `width × height` uint32 offsets — the linear source
pixel index for each destination pixel, or `0xFFFFFFFF` for out-of-bounds.

This makes the C remap a single indexed read per pixel with no float math.

In [ ]:
def _write_remap_bin(path, width, height, map1_f32, map2_f32):
    """Export a pre-computed pixel-offset table for the C remap loader.

    map1_f32 and map2_f32 are the (x, y) coordinate maps from
    cv2.initUndistortRectifyMap with CV_32FC1.
    """
    src_x = np.round(map1_f32).astype(np.int32)
    src_y = np.round(map2_f32).astype(np.int32)
    valid = (src_x >= 0) & (src_x < width) & (src_y >= 0) & (src_y < height)
    offsets = np.where(valid, src_y * width + src_x, 0xFFFFFFFF).astype(np.uint32)

    with open(path, "wb") as f:
        f.write(b"RMAP")
        np.array([width, height, 0], dtype=np.uint32).tofile(f)
        offsets.tofile(f)

    size_mb = os.path.getsize(path) / (1024 * 1024)
    print(f"  {os.path.basename(path)}: {width}x{height}, {size_mb:.1f} MB")

lbin = os.path.join(calib_result_path, "remap_left.bin")
rbin = os.path.join(calib_result_path, "remap_right.bin")

print("Exporting C-ready remap tables:")
_write_remap_bin(lbin, image_size[0], image_size[1], map_l1, map_l2)
_write_remap_bin(rbin, image_size[0], image_size[1], map_r1, map_r2)
print(f"\nUse with: ag-cam-tools stream --rectify {session_path}")

## Inspect rectified output

Visual sanity check — apply the rectification maps to the last image pair.
Corresponding points should fall on the same horizontal scanline in both
rectified images.  The quantitative epipolar error measured above should
confirm this; this plot is a quick eyeball check.

In [ ]:
import matplotlib.pyplot as plt

rect_left  = cv2.remap(img_left,  map_l1, map_l2, cv2.INTER_LINEAR)
rect_right = cv2.remap(img_right, map_r1, map_r2, cv2.INTER_LINEAR)

unrectified = np.hstack((
    cv2.cvtColor(img_left,  cv2.COLOR_BGR2RGB),
    cv2.cvtColor(img_right, cv2.COLOR_BGR2RGB),
))
rectified = np.hstack((
    cv2.cvtColor(rect_left,  cv2.COLOR_BGR2RGB),
    cv2.cvtColor(rect_right, cv2.COLOR_BGR2RGB),
))

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

for ax, img, title in [
    (axes[0], unrectified, "Original stereo pair (left | right)"),
    (axes[1], rectified,   "Rectified stereo pair (left | right)"),
]:
    ax.imshow(img)
    h = img.shape[0]
    for y in range(0, h, h // 16):
        ax.axhline(y=y, color="lime", linewidth=0.5, alpha=0.6)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()

print("Open 3.Depthmap_with_Tuning_Bar.ipynb to continue.")

## Disparity range estimation

For classical stereo matchers (StereoBM, StereoSGBM) you must set
`numDisparities` and `minDisparity`.  These depend on the baseline, focal
length, and working distance range.

Set `z_near` and `z_far` below to your expected operating range (same units
as `square_size` — cm by default), and this cell will compute the
recommended matcher parameters.

In [ ]:
# Working distance range (same units as square_size, i.e. cm).
z_near = 30    # closest expected object distance
z_far  = 200   # farthest expected object distance

f_px = P1[0, 0]                    # rectified focal length in pixels
baseline = np.linalg.norm(T)       # baseline in cm (same units as square_size)

d_max = f_px * baseline / z_near   # disparity at near plane
d_min = f_px * baseline / z_far    # disparity at far plane

# numDisparities must be a multiple of 16 and cover d_max.
num_disp = int(np.ceil(d_max / 16) * 16)
min_disp = max(0, int(np.floor(d_min)))

print(f"Rectified focal length : {f_px:.1f} px")
print(f"Baseline               : {baseline:.2f} cm")
print(f"Working range          : {z_near}–{z_far} cm")
print()
print(f"Disparity at z_near={z_near} cm : {d_max:.1f} px")
print(f"Disparity at z_far={z_far} cm  : {d_min:.1f} px")
print()
print(f"Recommended StereoBM / StereoSGBM parameters:")
print(f"  minDisparity    = {min_disp}")
print(f"  numDisparities  = {num_disp}")
print()
print(f"Depth resolution at z_far ({z_far} cm): "
      f"{baseline * f_px / (d_min + 1)**2:.2f} cm/disparity-step")

## Export calibration metadata

Write a machine-readable JSON summary of the calibration session.
Downstream tools can read this instead of loading individual `.npy` files
to get key parameters (baseline, focal length, image size, error metrics).

In [ ]:
import json

meta = {
    "image_size": list(image_size),
    "num_pairs_used": len(object_points),
    "checkerboard": {
        "rows": rows,
        "columns": columns,
        "square_size_cm": square_size,
    },
    "distortion_model": "rational_8",
    "rms_stereo_px": round(float(rms), 4),
    "rms_left_px": round(float(rms_l), 4),
    "rms_right_px": round(float(rms_r), 4),
    "per_pair_rms_px": [round(float(v), 4) for v in per_pair_rms_combined],
    "mean_epipolar_error_px": round(float(mean_epipolar), 4),
    "max_epipolar_error_px": round(float(max_epipolar), 4),
    "baseline_cm": round(float(np.linalg.norm(T)), 4),
    "focal_length_px": round(float(P1[0, 0]), 2),
    "principal_point_px": [round(float(P1[0, 2]), 2), round(float(P1[1, 2]), 2)],
    "disparity_range": {
        "z_near_cm": z_near,
        "z_far_cm": z_far,
        "min_disparity": min_disp,
        "num_disparities": num_disp,
    },
}

meta_path = os.path.join(calib_result_path, "calibration_meta.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print(f"Metadata written to {meta_path}")
print(json.dumps(meta, indent=2))